# Onboarding as a Data Owner

This notebook is the Data Owner view of part 6. You should have already run through steps 1-7 in the OM Authority notebook 6. If you have not done so proceed to port 8888 (either localhost or aws) and complete the steps in notebook `6. Onboarding a Data Owner` first.



## 8. Initialising the Data Owner Controller

This is very similar to the code used for the OM Authority accept the arguments used to configure the AriesAgentController are different.

The point to the ports and webhook location used when starting the data-scientist-agent ACA-Py instance. You can see these arguments in the `manage` script line 145.


In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    

ADMIN_URL = "http://dataowner-agent:8051"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""

agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE)

IPython autoawait is `on`, and set to use `asyncio`


## 9. Start Webhook Server and Register Listeners

The cred handler webhook has changed to held you understand the different aspects of the issue-credential protocol and get a deeper appreciation of the complex cryptography that is handled under the hood.

You do not need to review this code, but be sure to pay attention to the output produced when the data owner is issued a credential.

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())


def cred_handler(payload):
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n----------------------------------------------\n")
    print("Handle Credential Webhook Payload")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n----------------------------------------------\n")



    
    # Note the Data Owner is acting as the holder in this notebook
    if role == "holder":
        if state == "offer_received":
            print("The payload contains a proposal dictional and a credential_offer object used for the crypto\n")
            proposal = payload["credential_proposal_dict"]
            print("The proposal dictionary is likely how you would understand and display a credential offer in your application")
            print("\n", proposal)
            print("\n This includes the set of attributes you are being offered")
            attributes = proposal['credential_proposal']['attributes']
            print(attributes)
            print("\nA credential_offer contains cryptographic proof that the issuer controls a set of key associated with a schema and credential definition identifier")
            print(f"\nCredential offer for schema id {payload['credential_offer']['schema_id']} and cred def id {payload['credential_offer']['cred_def_id']}")
            print("\nKey correctness proof - some big math numbers that can be used by the crypto.\n")
            print(payload["credential_offer"]["key_correctness_proof"])
            print("\nThe credential offer also includes a nonce, the holder needs to use this when creating a commitment to their master secret so the issuer has confidence that they created the commitment in that moment")
            print("\nNonce",payload["credential_offer"]["nonce"])
        if state == "request_sent":
            print("\nA credential request object contains the commitment to the agents master secret using the nonce from the offer")
            
            credential_request = payload["credential_request"]
            print("\n", credential_request)
            print("\n")
            print("Blinded Master Secret - notice the hidden attributes property\n")
            print(credential_request["blinded_ms"])
            print("\nZKP proof of correct blinding\n")
            print(credential_request["blinded_ms_correctness_proof"])
            
            print("\n You don't need to worry about these values usually, all the crypto happens under the hood. But it's pretty cool to see it's there.\n")
            
            
        if state == "credential_received":
            raw_cred = payload["raw_credential"]
            print("The raw_credential contains the cryptographic signature from the issuer \n")
            print(raw_cred)
            print("\n")
            print("It can be a bit overwhelming, lets go through some key elements \n")
            print("Values are the attributes you where issued identified in the dictionary by the attribute names from the schema. The values are shown in both the raw (text) and the encoded format used for the signature")
            print("\n",raw_cred["values"])
            print("\n")
            print("The Signature, using this object your agent can create unlinkable presentations any subset of attributes in this credential\n")
            print(raw_cred["signature"])
            print("\n")
            print("Notice the p_credential and r_credential, the r_credential stands for revocation and should be empty because we haven't issued a revocable credential yet")
            print("Lets look at the primary p_credential in more detail\n")
            primary_cred = raw_cred["signature"]["p_credential"]
            print(primary_cred)
            print("It contains an ecoding of your blinded master secret (I think) : ", primary_cred["m_2"])
            print("\nThe remaining a, e, v are all elements of the signature")
            print("\na :", primary_cred["a"])
            print("\ne :", primary_cred["e"])
            print("\n v:", primary_cred["v"])
            print("\n")
            print("These appear to map well to the CL02 paper on the crypto, with 'a' being used instead of 's'\n")
            
            print("Signature Correctness Proof - gives holder cryptographic assurance the signature is correct\n")
            print(raw_cred["signature_correctness_proof"])
            
            print("\nNotice how all correctness proofs we have seen have an argument - c. This is the challenge part of a non-interactive ZKP proof. You will learn about this in more detail in the ZKP part of the course.")
            
        if state == "credential_acked":
            credential = payload["credential"]
            print("The Credential Object\n")
            print(credential)
            print("\nThe referent acts as the identifier for retrieving the raw credential from the wallet")
            print("Referent", credential["referent"])
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    print("Handle Connection Webhook Payload")
    connection_id = payload["connection_id"]
    print("Connection ID", connection_id)

    state = payload['state']
    print("State", state)
    if state == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)

Subscribing too: issue_credential
Subscribing too: connections


## 10. Copy Invitation from OM Authority

You should have **copied** the invitation object output from step 7. in the OM Authority notebook. **First** paste that over the example object shown below, **then** run the cell.

In [3]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'fd1bd5bc-5d27-422f-bdaa-ac44625c145a', 'serviceEndpoint': 'https://b1584135fc27.ngrok.io', 'recipientKeys': ['CY59xQ1ToVqPJX6hj1fqCJzR4EshrSyjpfPA2ysMap9r'], 'label': 'OM_AUTHORITY'}


## 11. Accept the Connection

**Be sure to scroll through the extended output that digs into the content of the messages communicated between issuer and holder at each stage of the protocol**

Note the CL02 paper can be found [here](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.186.5994&rep=rep1&type=pdf) for the inquisitive

In [4]:
await agent_controller.connections.accept_connection(invitation)

wallet None
Handle Connection Webhook Payload
Connection ID 940256c1-9e11-47da-a576-399865939654
State invitation
wallet None
Handle Connection Webhook Payload
Connection ID 940256c1-9e11-47da-a576-399865939654
State request


{'state': 'request',
 'created_at': '2021-04-06 13:25:58.084537Z',
 'their_role': 'inviter',
 'invitation_key': 'CY59xQ1ToVqPJX6hj1fqCJzR4EshrSyjpfPA2ysMap9r',
 'routing_state': 'none',
 'updated_at': '2021-04-06 13:25:58.110688Z',
 'connection_id': '940256c1-9e11-47da-a576-399865939654',
 'my_did': 'W8jzDB48xZuYrr1nmD3izC',
 'request_id': '170683e2-56ba-453b-8d3d-00a9b9a100a6',
 'invitation_mode': 'once',
 'rfc23_state': 'request-sent',
 'their_label': 'OM_AUTHORITY',
 'accept': 'manual'}

wallet None
Handle Connection Webhook Payload
Connection ID 940256c1-9e11-47da-a576-399865939654
State response
wallet None
Handle Connection Webhook Payload
Connection ID 940256c1-9e11-47da-a576-399865939654
State active
Connection 940256c1-9e11-47da-a576-399865939654 changed state to active
wallet None

----------------------------------------------

Handle Credential Webhook Payload
Credential exchange ID : fcb40dfb-aed4-4c89-b7b1-320716f60b0d
Agent Protocol Role :  holder
Protocol State :  offer_received

----------------------------------------------

The payload contains a proposal dictional and a credential_offer object used for the crypto

The proposal dictionary is likely how you would understand and display a credential offer in your application

 {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': '9f87b7cc-03d0-4365-b487-00ddc902e852', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/crede

## 12. Review the output from the webhook listeners in the OM Authority notebook

The state from the issue-credential webhook payloads should be slightly different.

Don't forget to terminate the AriesAgentController in this tutorial

In [5]:
await agent_controller.terminate()

## Continue to Notebook 7.

Part 7 does not involve the OM Authority. You will establish a connection with the data scientist and mutually authenticate each other by requesting a proof of a credential.

First make sure you have terminated the controller in the OM Authority notebook.